# Validating and Importing Item Metadata <a class="anchor" id="top"></a>

In this notebook, you will pick up where you left off in `01_Validating_and_Importing_User_Item_Interaction_Data.ipynb` to build a working item metadata dataset. This will allow you to work with filters as well as later support the `User Personalization` or `HRNN-Metadata` algorithms.


To run this notebook, you need to have run the previous notebook, `01_Validating_and_Importing_User_Item_Interaction_Data`, where you created a dataset and imported interaction data into Amazon Personalize. At the end of that notebook, you saved some of the variable values, which you now need to load into this notebook.

In [ ]:
%store -r

## Prepare your Item metadata <a class="anchor" id="prepare"></a>
[Back to top](#top)

The next thing to be done is to load the data and confirm the data is in a good state, then save it to a CSV where it is ready to be used with Amazon Personalize.

To get started, import a collection of Python libraries commonly used in data science.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

Next,open the data file and take a look at the first several rows.

In [ ]:
original_data = pd.read_csv(dataset_dir + '/movies.csv')
original_data.head(5)

In [ ]:
original_data.describe()

This does not really tell us much about the dataset, so we will explore a bit more for just raw info. We can see that genres are often grouped together, and that is fine for us as Personalize does support this structure.

In [ ]:
original_data.info()

From this, you can see that there are a total of (62,000+ for full 9742 for small) entries in the dataset, with 3 columns.

This is a pretty minimal dataset of just the movieId, title and the list of genres that are applicable to each entry. However there is additional data available in the Movielens dataset. For instance the title includes the year of the movies release. Let's make that another column of metadata

In [ ]:
original_data['year'] =original_data['title'].str.extract('.*\((.*)\).*',expand = False)
original_data.head(5)

From an item metadata perspective, we only want to include information that is relevant to training a model and/or filtering resulte, so we will drop the title, retaining the genre information.

In [ ]:
itemmetadata_df = original_data.copy()
itemmetadata_df = itemmetadata_df[['movieId', 'genres', 'year']]
itemmetadata_df.head()

After manipulating the data, always confirm if the data format has changed.

In [ ]:
itemmetadata_df.dtypes

Amazon Personalize has a default column for `ITEM_ID` that will map to our `movieId`, and now we can flesh out more information by specifying `GENRE` as well.

In [ ]:
itemmetadata_df.rename(columns = {'genres':'GENRE', 'movieId':'ITEM_ID', 'year':'YEAR'}, inplace = True) 

That's it! At this point the data is ready to go, and we just need to save it as a CSV file.

In [ ]:
itemmetadata_filename = "item-meta.csv"
itemmetadata_df.to_csv((data_dir+"/"+itemmetadata_filename), index=False, float_format='%.0f')

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Create the dataset

First, define a schema to tell Amazon Personalize what type of dataset you are uploading. There are several reserved and mandatory keywords required in the schema, based on the type of dataset. More detailed information can be found in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Here, you will create a schema for item metadata data, which needs the `ITEM_ID` and `GENRE` fields. These must be defined in the same order in the schema as they appear in the dataset.

In [ ]:
itemmetadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string",
            "categorical": True
        },{
            "name": "YEAR",
            "type": "int",
        },
        
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-poc-movielens-item",
    schema = json.dumps(itemmetadata_schema)
)

itemmetadataschema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

With a schema created, you can create a dataset within the dataset group. Note, this does not load the data yet. This will happen a few steps later.

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-poc-movielens-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = itemmetadataschema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Upload data to S3

Now that your Amazon S3 bucket has been created, upload the CSV file of our user-item-interaction data. 

In [ ]:
itemmetadata_file_path = data_dir + "/" + itemmetadata_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(itemmetadata_filename).upload_file(itemmetadata_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+itemmetadata_filename

## Import the item metadata <a class="anchor" id="import"></a>
[Back to top](#top)

Earlier you created the dataset group and dataset to house your information, so now you will execute an import job that will load the data from the S3 bucket into the Amazon Personalize dataset. 

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-item-import1",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, itemmetadata_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

Before we can use the dataset, the import job must be active. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the import job every second, up to a maximum of 6 hours.

Importing the data can take some time, depending on the size of the dataset. In this workshop, the data import job should take around 15 minutes.

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

With this import now complete you can enable filtering for your recommendations as well as support `HRNN-Metadata`. Run the cell below before moving on to store a few values for usage in the next notebooks. After completing that cell open notebook `03_Creating_and_Evaluating_Solutions.ipynb` to continue.

In [ ]:
%store items_dataset_arn
%store itemmetadataschema_arn